In [1]:
!pip install scikeras[tensorflow]

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor

# Loading the dataset
data = pd.read_csv('../data/AmesData.csv')

# Filling NaN values in numeric columns with the median
for col in data.select_dtypes(include=[np.number]).columns:
    data[col].fillna(data[col].median(), inplace=True)

# Filling NaN values in categorical columns with the most frequent value
for col in data.select_dtypes(include=[object]).columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Encoding categorical variables using one-hot encoding
data_encoded = pd.get_dummies(data, drop_first=True)

# Splitting the data into features and target
X = data_encoded.drop('SalePrice', axis=1)
y = data['SalePrice']

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing the scaler for the features
scaler = StandardScaler()

# Scaling the features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Scaling the target variable
y_train_mean = y_train.mean()
y_train_std = y_train.std()
y_train_scaled = (y_train - y_train_mean) / y_train_std
y_test_scaled = (y_test - y_train_mean) / y_train_std

# Defining a function to create the model (for KerasRegressor)
def create_model(optimizer='adam', learning_rate=0.001, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Creating the KerasRegressor
model = KerasRegressor(model=create_model, epochs=100, batch_size=32, verbose=0)

# Defining the hyperparameters grid
param_grid = {
    'model__optimizer': ['adam'],
    'model__learning_rate': [0.01, 0.001, 0.0001],
    'model__dropout_rate': [0.0, 0.3, 0.5],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100]
}

# Performing RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, verbose=1)
random_search_result = random_search.fit(X_train_scaled, y_train_scaled)

print(f"Best: {random_search_result.best_score_} using {random_search_result.best_params_}")

# Evaluating the best model on the test set
best_model = random_search_result.best_estimator_
mae_scaled = best_model.score(X_test_scaled, y_test_scaled)
print(f'Mean Absolute Error on scaled data: {mae_scaled}')

# Predicting on the test data
predictions_scaled = best_model.predict(X_test_scaled)

# Rescaling the predictions to the original scale
predictions_rescaled = predictions_scaled * y_train_std + y_train_mean
y_test_rescaled = y_test

# Calculating the rescaled mean absolute error
mae_rescaled = mean_absolute_error(y_test_rescaled, predictions_rescaled)
print(f'Rescaled Mean Absolute Error: {mae_rescaled}')

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.3 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.0 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.13-py3-none-any.whl size=1518705 sha256=f1d02552a5e357681bcaab262d063c12bf8dcbbfa16daa794a4e97376772e555
  Stored in directory: /home/vscode/.cache/pip/wh

2024-07-29 19:06:35.916957: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 19:06:35.955385: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 19:06:35.957278: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 19:06:37.345996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best: 0.8452076237345957 using {'model__optimizer': 'adam', 'model__learning_rate': 0.001, 'model__dropout_rate': 0.3, 'epochs': 50, 'batch_size': 64}
Mean Absolute Error on scaled data: 0.9159975888259128
Rescaled Mean Absolute Error: 14853.925471006318
